In [ ]:
import os
import sys
import pickle

projectabspathname = os.path.abspath('projectname.pickle')
print(projectabspathname)
projectname = 'modelizacion.ipynb'
projectpickle = open(str(projectabspathname),'wb')
pickle.dump(projectname, projectpickle)
projectpickle.close()

/content/projectname.pickle


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
import math
import tensorflow as tf

In [ ]:
url1="https://raw.githubusercontent.com/irl01-umh/TFG/main/train_peptides_limpio.csv"
url2="https://raw.githubusercontent.com/irl01-umh/TFG/main/train_proteins.csv"
url3="https://raw.githubusercontent.com/irl01-umh/TFG/main/train_clinical_data.csv"
#Carga de train_peptides.csv, train_proteins.csv y train_clinical_data.csv
train_peptides = pd.read_csv(url1)
train_peptides.head()

train_proteins = pd.read_csv(url2)
train_proteins.head()

train_clinical_data = pd.read_csv(url3)
train_clinical_data.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [ ]:
# MONTAR DRIVE
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Ficheros csv base
url_pep = '/content/drive/Shareddrives/TFGs - asun.mayoral@umh.es/INMA RAMÍREZ 23-24/train_peptides_limpio.csv'
url_clin = '/content/drive/Shareddrives/TFGs - asun.mayoral@umh.es/INMA RAMÍREZ 23-24/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv'
url_prot = '/content/drive/Shareddrives/TFGs - asun.mayoral@umh.es/INMA RAMÍREZ 23-24/amp-parkinsons-disease-progression-prediction/train_proteins.csv'

train_peptides = pd.read_csv(url_pep)
train_clinical_data = pd.read_csv(url_clin)
train_proteins = pd.read_csv(url_prot)

In [ ]:
# obtenido de gusthema. Ver explicación sobre el combinado de datos
# Function to prepare dataset with all the steps mentioned above:
def prepare_dataset(train_proteins, train_peptides):
    # Step 1: Grouping
    df_protein_grouped = train_proteins.groupby(['visit_id','UniProt'])['NPX'].mean().reset_index()
    df_peptide_grouped = train_peptides.groupby(['visit_id','Peptide_dep'])['PeptideAbundance'].mean().reset_index()

    # Step 2: Pivoting
    df_protein = df_protein_grouped.pivot(index='visit_id',columns = 'UniProt', values = 'NPX').rename_axis(columns=None).reset_index()
    df_peptide = df_peptide_grouped.pivot(index='visit_id',columns = 'Peptide_dep', values = 'PeptideAbundance').rename_axis(columns=None).reset_index()

    # Step 3: Merging
    #pro_pep_df = df_protein.merge(df_peptide, on = ['visit_id'], how = 'left')
    pro_pep_df = df_protein.merge(df_peptide, on = ['visit_id'], how = 'left')


    return pro_pep_df

In [ ]:
pro_pep_df=prepare_dataset(train_proteins, train_peptides)

In [ ]:
pro_pep_df.shape

(1113, 1196)

In [ ]:
# Display the first 5 examples
pro_pep_df.head(5)

,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVMLPVADQDQCaIR,YVMbLPVADQDQCaIR,YVNKEIQNAVNGVK,YWGVASFLQK,YYCaFQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,...,202274.0,NaN,4401830.0,583075.0,77482.6,76705.7,104260.0,530223.0,NaN,7207.30
1,10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,...,201009.0,NaN,5001750.0,355643.0,36745.3,92078.1,123254.0,453883.0,49281.9,25332.80
2,10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,...,220728.0,NaN,5424380.0,496021.0,39016.0,63203.6,128336.0,447505.0,52389.1,21235.70
3,10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,...,188362.0,9433.71,3900280.0,328482.0,48210.3,89822.1,129964.0,552232.0,65657.8,9876.98
4,10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,...,206187.0,6365.15,3521800.0,496737.0,69984.6,80919.3,111799.0,NaN,56977.6,4903.09


In [ ]:
from sklearn.impute import SimpleImputer

columns_with_nan_pro_pep = [col for col in pro_pep_df.columns if pro_pep_df[col].isnull().any()]

imputer_pro_pep = SimpleImputer(strategy='mean')
pro_pep_df_imputed = pro_pep_df.copy()
pro_pep_df_imputed[columns_with_nan_pro_pep] = imputer_pro_pep.fit_transform(pro_pep_df[columns_with_nan_pro_pep])

imputer_clinical = SimpleImputer(strategy='mean')
train_clinical_data_imputed = train_clinical_data.copy()
train_clinical_data_imputed[['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']] = imputer_clinical.fit_transform(train_clinical_data[['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']])

train_updrs_pre = pro_pep_df_imputed.merge(train_clinical_data_imputed[['visit_id', 'patient_id', 'visit_month', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']], on='visit_id', how='left')
imputer_train=SimpleImputer(strategy='mean')
columns_with_nan = [col for col in train_updrs_pre.columns if train_updrs_pre[col].isnull().any()]
train_updrs=train_updrs_pre.copy()
train_updrs[columns_with_nan] = imputer_train.fit_transform(train_updrs_pre[columns_with_nan])

print(train_updrs)

#train_updrs.to_csv('train_updrs.csv', index=False)


      visit_id        O00391    O00533        O00584        O14498  \
0      10053_0   9104.270000  402321.0  26505.529157  27305.934884   
1     10053_12  10464.200000  435586.0  26505.529157  27305.934884   
2     10053_18  13235.700000  507386.0   7126.960000  24525.700000   
3     10138_12  12600.200000  494581.0   9165.060000  27193.500000   
4     10138_24  12003.200000  522138.0   4498.510000  17189.800000   
...        ...           ...       ...           ...           ...   
1108   8699_24   9983.000000  400290.0  24240.100000  27305.934884   
1109    942_12   6757.320000  360858.0  18367.600000  14760.700000   
1110    942_24  11641.264435  352722.0  22834.900000  23393.100000   
1111    942_48  11627.800000  251820.0  22046.500000  26360.500000   
1112     942_6  11218.700000  399518.0  20581.000000  31290.900000   

            O14773       O14791    O15240   O15394    O43505  ...  YWGVASFLQK  \
0      7150.570000  2497.840000   83002.9  15113.6  167327.0  ...    104260.0 

In [ ]:
if train_updrs.isnull().any().any():
    print("El DataFrame train_updrs tiene valores NaN.")
else:
    print("El DataFrame train_updrs no tiene valores NaN.")


El DataFrame train_updrs no tiene valores NaN.


In [ ]:
train_updrs['updrs_1'].mean()

6.5664794007490634

In [ ]:
train_updrs['updrs_2'].mean()

5.821161048689139

In [ ]:
train_updrs['updrs_3'].mean()

17.30621051448069

In [ ]:
train_updrs['updrs_4'].mean()

1.4309846810293994

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV


In [ ]:
def train_models(pro_pep_df, train_clinical_data, name):
    X = train_updrs.drop(['visit_id', name], axis=1)
    y = train_updrs[name]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    final_models = {
        'name': [],
        'model': [],
        'root_mean_sqr': [],
        'absolute_mean': [],
        'r2_score': []
    }

    models = {
        "Linear Regression": LinearRegression(),
        "Lasso": Lasso(),
        "Ridge": Ridge(),
        "K-Neighbors Regressor": KNeighborsRegressor(),
        "Decision Tree": DecisionTreeRegressor(),
        "Random Forest Regressor": RandomForestRegressor()
    }

    for model_name, model in models.items():
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('feature_selection', SelectKBest(f_regression)),
            ('model', model)
        ])

        param_grid = {}
        if model_name == "Linear Regression":
            param_grid = {
                'feature_selection__k': [5, 10, 15, 20]
            }
        elif model_name in ["Lasso", "Ridge"]:
            param_grid = {
                'feature_selection__k': [5, 10, 15, 20],
                'model__alpha': [0.1, 1, 10, 100]
            }
        elif model_name == "K-Neighbors Regressor":
            param_grid = {
                'feature_selection__k': [5, 10, 15, 20],
                'model__n_neighbors': [5, 10, 15, 20]
            }
        elif model_name == "Decision Tree":
            param_grid = {
                'feature_selection__k': [5, 10, 15, 20],
                'model__max_depth': [5, 10, 15, 20]
            }
        elif model_name == "Random Forest Regressor":
            param_grid = {
                'feature_selection__k': [5, 10, 15, 20],
                'model__n_estimators': [100, 200, 300, 400]
            }

        grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)

        model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_pred)

        final_models['name'].append(model_name)
        final_models['model'].append(best_model)
        final_models['root_mean_sqr'].append(model_test_rmse)
        final_models['absolute_mean'].append(model_test_mae)
        final_models['r2_score'].append(model_test_r2)

    return pd.DataFrame(final_models).sort_values('r2_score', ascending=False)


def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


def best_model(pro_pep_df, train_clinical_data, name):
    return train_models(pro_pep_df, train_clinical_data, name).iloc[0]['model']

In [ ]:
mejor_modelo_1 = best_model(pro_pep_df, train_clinical_data, 'updrs_1')
print("El mejor modelo para predecir '{}' es: {}".format('updrs_1', mejor_modelo_1))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


El mejor modelo para predecir 'updrs_1' es: Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection',
                 SelectKBest(k=20,
                             score_func=<function f_regression at 0x7b2fcf3ffbe0>)),
                ('model', RandomForestRegressor(n_estimators=200))])


In [ ]:
results_df = train_models(pro_pep_df, train_clinical_data, 'updrs_1')
print(results_df)

                      name                                              model  \
5  Random Forest Regressor  (StandardScaler(), SelectKBest(k=20, score_fun...   
0        Linear Regression  (StandardScaler(), SelectKBest(k=5, score_func...   
2                    Ridge  (StandardScaler(), SelectKBest(k=5, score_func...   
1                    Lasso  (StandardScaler(), SelectKBest(k=15, score_fun...   
3    K-Neighbors Regressor  (StandardScaler(), SelectKBest(score_func=<fun...   
4            Decision Tree  (StandardScaler(), SelectKBest(score_func=<fun...   

   root_mean_sqr  absolute_mean  r2_score  
5       3.665020       2.741209  0.540929  
0       3.702628       2.799548  0.531459  
2       3.703085       2.800001  0.531344  
1       3.789158       2.871303  0.509304  
3       3.946162       3.030328  0.467797  
4       4.050800       2.961656  0.439199  


In [ ]:
mejor_modelo_2 = best_model(pro_pep_df, train_clinical_data, 'updrs_2')
print("El mejor modelo para predecir '{}' es: {}".format('updrs_2', mejor_modelo_2))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


El mejor modelo para predecir 'updrs_2' es: Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection',
                 SelectKBest(score_func=<function f_regression at 0x7be944942200>)),
                ('model', RandomForestRegressor(n_estimators=400))])


In [ ]:
results_df2 = train_models(pro_pep_df, train_clinical_data, 'updrs_2')
print(results_df2)

                      name                                              model  \
5  Random Forest Regressor  (StandardScaler(), SelectKBest(score_func=<fun...   
2                    Ridge  (StandardScaler(), SelectKBest(score_func=<fun...   
0        Linear Regression  (StandardScaler(), SelectKBest(score_func=<fun...   
1                    Lasso  (StandardScaler(), SelectKBest(score_func=<fun...   
3    K-Neighbors Regressor  (StandardScaler(), SelectKBest(k=5, score_func...   
4            Decision Tree  (StandardScaler(), SelectKBest(k=5, score_func...   

   root_mean_sqr  absolute_mean  r2_score  
5       3.154986       2.311086  0.730453  
2       3.170295       2.325513  0.727831  
0       3.170302       2.325533  0.727830  
1       3.172909       2.302581  0.727382  
3       3.308448       2.234820  0.703594  
4       3.444939       2.455907  0.678632  


In [ ]:
mejor_modelo_3 = best_model(pro_pep_df, train_clinical_data, 'updrs_3')
print("El mejor modelo para predecir '{}' es: {}".format('updrs_3', mejor_modelo_3))

El mejor modelo para predecir 'updrs_3' es: Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection',
                 SelectKBest(k=20,
                             score_func=<function f_regression at 0x7be944942200>)),
                ('model', RandomForestRegressor(n_estimators=400))])


In [ ]:
results_df3 = train_models(pro_pep_df, train_clinical_data, 'updrs_3')
print(results_df3)

                      name                                              model  \
5  Random Forest Regressor  (StandardScaler(), SelectKBest(k=20, score_fun...   
1                    Lasso  (StandardScaler(), SelectKBest(k=20, score_fun...   
2                    Ridge  (StandardScaler(), SelectKBest(k=20, score_fun...   
0        Linear Regression  (StandardScaler(), SelectKBest(k=20, score_fun...   
3    K-Neighbors Regressor  (StandardScaler(), SelectKBest(k=5, score_func...   
4            Decision Tree  (StandardScaler(), SelectKBest(k=5, score_func...   

   root_mean_sqr  absolute_mean  r2_score  
5       9.152557       6.599613  0.643975  
1       9.853456       7.656278  0.587359  
2       9.913745       7.674270  0.582294  
0       9.915885       7.674875  0.582113  
3       9.940497       7.431432  0.580036  
4      10.516043       7.349827  0.529997  


In [ ]:
mejor_modelo_4 = best_model(pro_pep_df, train_clinical_data, 'updrs_4')
print("El mejor modelo para predecir '{}' es: {}".format('updrs_4', mejor_modelo_4))

El mejor modelo para predecir 'updrs_4' es: Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x7be944942200>)),
                ('model', KNeighborsRegressor(n_neighbors=20))])


In [ ]:
results_df4 = train_models(pro_pep_df, train_clinical_data, 'updrs_4')
print(results_df4)

                      name                                              model  \
3    K-Neighbors Regressor  (StandardScaler(), SelectKBest(k=5, score_func...   
5  Random Forest Regressor  (StandardScaler(), SelectKBest(score_func=<fun...   
1                    Lasso  (StandardScaler(), SelectKBest(k=5, score_func...   
0        Linear Regression  (StandardScaler(), SelectKBest(k=5, score_func...   
2                    Ridge  (StandardScaler(), SelectKBest(k=5, score_func...   
4            Decision Tree  (StandardScaler(), SelectKBest(k=5, score_func...   

   root_mean_sqr  absolute_mean  r2_score  
3       1.912297       1.293864  0.201510  
5       1.931823       1.315595  0.185121  
1       1.971000       1.370874  0.151734  
0       1.978535       1.402324  0.145236  
2       1.980519       1.393496  0.143521  
4       2.592911       1.492059 -0.468026  


In [ ]:
pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.2 MB/s eta 0:00:00


In [ ]:
import os
os.environ["LOKY_BACKEND"] = "threading"

# Luego importas joblib
import joblib

In [ ]:
#CATBOOSTING
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
import pandas as pd

def train_catboost_model(pro_pep_df, train_clinical_data, name):
    X = train_updrs.drop(['visit_id', name], axis=1)
    y = train_updrs[name]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    final_models = {
        'name': [],
        'model': [],
        'root_mean_sqr': [],
        'absolute_mean': [],
        'r2_score': []
    }

    catboost_model = CatBoostRegressor()

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('feature_selection', SelectKBest(f_regression)),
        ('model', catboost_model)
    ])

    param_grid = {
        'feature_selection__k': [5, 10, 15, 20],
        'model__learning_rate': [0.01, 0.1, 0.5, 1],
        'model__depth': [3, 5, 7, 9],
        'model__n_estimators': [100, 200, 300, 400]
    }

    grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_pred)

    final_models['name'].append("CatBoostRegressor")
    final_models['model'].append(best_model)
    final_models['root_mean_sqr'].append(model_test_rmse)
    final_models['absolute_mean'].append(model_test_mae)
    final_models['r2_score'].append(model_test_r2)

    return pd.DataFrame(final_models).sort_values('r2_score', ascending=False)


def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square



In [ ]:
results_cat1 = train_catboost_model(pro_pep_df, train_clinical_data, 'updrs_1')
print(results_cat1)

0:	learn: 5.0276843	total: 112ms	remaining: 33.5s
1:	learn: 4.8462003	total: 136ms	remaining: 20.3s
2:	learn: 4.6985373	total: 174ms	remaining: 17.3s
3:	learn: 4.5710493	total: 208ms	remaining: 15.4s
4:	learn: 4.4561852	total: 246ms	remaining: 14.5s
5:	learn: 4.3366173	total: 284ms	remaining: 13.9s
6:	learn: 4.2412463	total: 330ms	remaining: 13.8s
7:	learn: 4.1505218	total: 363ms	remaining: 13.2s
8:	learn: 4.0608132	total: 402ms	remaining: 13s
9:	learn: 3.9772234	total: 438ms	remaining: 12.7s
10:	learn: 3.8967178	total: 483ms	remaining: 12.7s
11:	learn: 3.8161524	total: 526ms	remaining: 12.6s
12:	learn: 3.7508165	total: 574ms	remaining: 12.7s
13:	learn: 3.6845459	total: 618ms	remaining: 12.6s
14:	learn: 3.6233908	total: 657ms	remaining: 12.5s
15:	learn: 3.5745241	total: 693ms	remaining: 12.3s
16:	learn: 3.5294792	total: 733ms	remaining: 12.2s
17:	learn: 3.4767037	total: 777ms	remaining: 12.2s
18:	learn: 3.4255003	total: 812ms	remaining: 12s
19:	learn: 3.3691736	total: 856ms	remaining: 

In [ ]:
results_cat2 = train_catboost_model(pro_pep_df, train_clinical_data, 'updrs_2')
print(results_cat2)

0:	learn: 5.4828704	total: 49ms	remaining: 4.85s
1:	learn: 5.2219765	total: 51.2ms	remaining: 2.51s
2:	learn: 4.9761396	total: 56.3ms	remaining: 1.82s
3:	learn: 4.7581594	total: 59.3ms	remaining: 1.42s
4:	learn: 4.5363083	total: 66.5ms	remaining: 1.26s
5:	learn: 4.3477477	total: 68.6ms	remaining: 1.07s
6:	learn: 4.1799255	total: 75.6ms	remaining: 1s
7:	learn: 4.0156024	total: 77.7ms	remaining: 894ms
8:	learn: 3.8981813	total: 80.9ms	remaining: 818ms
9:	learn: 3.7687715	total: 84.9ms	remaining: 764ms
10:	learn: 3.6534760	total: 87.8ms	remaining: 710ms
11:	learn: 3.5549783	total: 91.4ms	remaining: 670ms
12:	learn: 3.4577955	total: 94.7ms	remaining: 634ms
13:	learn: 3.3846187	total: 98.3ms	remaining: 604ms
14:	learn: 3.3165136	total: 102ms	remaining: 575ms
15:	learn: 3.2514459	total: 105ms	remaining: 552ms
16:	learn: 3.2038076	total: 109ms	remaining: 531ms
17:	learn: 3.1546306	total: 112ms	remaining: 510ms
18:	learn: 3.1199719	total: 115ms	remaining: 492ms
19:	learn: 3.0749073	total: 119m

In [ ]:
results_cat3 = train_catboost_model(pro_pep_df, train_clinical_data, 'updrs_3')
print(results_cat3)

0:	learn: 13.9355858	total: 473us	remaining: 46.9ms
1:	learn: 13.2144929	total: 887us	remaining: 43.5ms
2:	learn: 12.6336009	total: 1.28ms	remaining: 41.3ms
3:	learn: 12.0734655	total: 1.65ms	remaining: 39.6ms
4:	learn: 11.6192372	total: 2.01ms	remaining: 38.2ms
5:	learn: 11.2677865	total: 2.39ms	remaining: 37.5ms
6:	learn: 10.9146252	total: 2.75ms	remaining: 36.6ms
7:	learn: 10.5391428	total: 3.12ms	remaining: 35.9ms
8:	learn: 10.3061624	total: 3.5ms	remaining: 35.4ms
9:	learn: 10.0166035	total: 3.88ms	remaining: 34.9ms
10:	learn: 9.7999584	total: 4.25ms	remaining: 34.4ms
11:	learn: 9.6148987	total: 4.7ms	remaining: 34.5ms
12:	learn: 9.4341657	total: 5.19ms	remaining: 34.8ms
13:	learn: 9.3016214	total: 5.6ms	remaining: 34.4ms
14:	learn: 9.2101494	total: 5.98ms	remaining: 33.9ms
15:	learn: 9.1256622	total: 6.36ms	remaining: 33.4ms
16:	learn: 9.0320405	total: 6.75ms	remaining: 32.9ms
17:	learn: 8.9590449	total: 7.12ms	remaining: 32.4ms
18:	learn: 8.8791757	total: 7.5ms	remaining: 32ms
1

In [ ]:
results_cat4 = train_catboost_model(pro_pep_df, train_clinical_data, 'updrs_4')
print(results_cat4) m

0:	learn: 2.6365751	total: 72ms	remaining: 28.7s
1:	learn: 2.5860063	total: 114ms	remaining: 22.7s
2:	learn: 2.5321810	total: 140ms	remaining: 18.6s
3:	learn: 2.4920075	total: 166ms	remaining: 16.5s
4:	learn: 2.4642951	total: 192ms	remaining: 15.2s
5:	learn: 2.4220163	total: 218ms	remaining: 14.3s
6:	learn: 2.3779921	total: 244ms	remaining: 13.7s
7:	learn: 2.3394892	total: 269ms	remaining: 13.2s
8:	learn: 2.3018522	total: 298ms	remaining: 13s
9:	learn: 2.2687298	total: 324ms	remaining: 12.6s
10:	learn: 2.2315198	total: 349ms	remaining: 12.3s
11:	learn: 2.1945742	total: 374ms	remaining: 12.1s
12:	learn: 2.1557301	total: 400ms	remaining: 11.9s
13:	learn: 2.1272548	total: 425ms	remaining: 11.7s
14:	learn: 2.0997586	total: 450ms	remaining: 11.6s
15:	learn: 2.0681105	total: 475ms	remaining: 11.4s
16:	learn: 2.0400269	total: 503ms	remaining: 11.3s
17:	learn: 2.0188100	total: 529ms	remaining: 11.2s
18:	learn: 1.9853897	total: 554ms	remaining: 11.1s
19:	learn: 1.9545691	total: 586ms	remaining:

In [ ]:
!pip install leaspy
#!git clone https://gitlab.com/icm-institute/aramislab/disease-course-mapping-solutions.git

  Using cached leaspy-1.2.0-py3-none-any.whl (558 kB)
INFO: pip is looking at multiple versions of leaspy to determine which version is compatible with other requirements. This could take a while.
  Using cached leaspy-1.1.2-py3-none-any.whl (556 kB)
  Using cached leaspy-1.0.3-py3-none-any.whl (547 kB)
  Using cached leaspy-1.0.2-py3-none-any.whl (542 kB)
ERROR: Cannot install leaspy==1.0.2, leaspy==1.0.3, leaspy==1.1.2 and leaspy==1.2.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    leaspy 1.2.0 depends on torch<1.11 and >=1.5.1
    leaspy 1.1.2 depends on torch<1.7 and >=1.2.0
    leaspy 1.0.3 depends on torch<1.7 and >=1.2.0
    leaspy 1.0.2 depends on torch<1.5 and >=1.1.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-reso

In [ ]:
from leaspy import AlgorithmSettings, Data, Leaspy
from leaspy.datasets import Loader

ModuleNotFoundError: No module named 'leaspy'